In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

### Old synthetic generator

In [2]:
# import scipy
# import torch
# from synthetics.generate import generate_single_task_unipolar_lfs

# L, _, accs, Y, _ = generate_single_task_unipolar_lfs(100000, 10, beta=0.5, 
#     class_balance=[0.6, 0.4])
# L = scipy.sparse.csr_matrix(L, dtype=np.int8)
# Y = torch.tensor(Y, dtype=torch.short)

### New synthetic generator

In [3]:
from synthetics.generate import generate_single_task_unipolar

n = 1000
m = 20
k = 2
L, Y, metadata = generate_single_task_unipolar(
    n, m, k=k, acc=[0.6, 0.9], rec=[0.1, 0.2], 
    class_balance=[0.7, 0.3], lf_balance=None, seed=1)

In [4]:
from metal.metrics import accuracy_score, coverage_score

# accs = metadata['accs']
# emp_accs = []
# for j in range(m):
#     emp_accs.append(accuracy_score(Y, L[:,j], ignore_in_pred=[0]))
# for a, b in zip(accs, emp_accs):
#     print(f"{a:.4f}, {b:.4f}, {abs(a - b):.8f}")

# recs = metadata['recs']
# polarities = metadata['polarities']
# emp_recs = []
# for j in range(m):
#     other_labels = [x for x in range(k + 1) if x != polarities[j]]
#     emp_recs.append(coverage_score(Y, L[:,j], ignore_in_gold=other_labels))
#     for a, b in zip(recs, emp_recs):
#         print(f"{a:.4f}, {b:.4f}, {abs(a - b):.8f}")


In [5]:
from metal.label_model.baselines import (
    RandomVoter, 
    MajorityLabelVoter, 
    MajorityClassVoter,
)

In [6]:
rv = RandomVoter()
rv.train(L)
# rv.predict(L)

In [7]:
scores = []
for i in range(100):
    score = rv.score(L, Y, verbose=False)
    scores.append(score)
print(np.mean(scores))

0.49834


In [8]:
from collections import Counter
counter = Counter(Y.numpy())
class_counts = np.array([v for k, v in sorted(counter.items())])
balances = [class_counts / sum(class_counts)]

mc = MajorityClassVoter()
mc.train(L, balances)
# mc.predict(L)

In [9]:
scores = []
for i in range(100):
    score = mc.score(L, Y, verbose=False)
    scores.append(score)
print(np.mean(scores))

0.7000000000000002


In [10]:
mv = MajorityLabelVoter()
mv.train(L)
# mv.predict(L)

In [11]:
scores = []
for i in range(100):
    score = mv.score(L, Y, verbose=False)
    scores.append(score)
print(np.mean(scores))

0.7920400000000001
